In [37]:
import numpy as np
import cvxpy as cp
import heapdict
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
import time

import open3d as o3d
import copy

In [38]:
def pc_to_np(open3d_data):
    return np.asarray(open3d_data.points)

def np_to_pc(np_data):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np_data)
    return pcd

In [43]:
def icp(refA, refB, T_init):
    source = o3d.io.read_point_cloud(refA, format='xyz')
    target = o3d.io.read_point_cloud(refB, format='xyz')
    
    threshold = 0.8
    
    print("Initial alignment")
    evaluation = o3d.registration.evaluate_registration(source, target, threshold, T_init)
    print(evaluation)

    print("Apply point-to-point ICP")
    reg_p2p = o3d.registration.registration_icp(source, target, threshold, T_init, 
                                                o3d.registration.TransformationEstimationPointToPoint())
    print(reg_p2p)
    print("Transformation is:")
    print(reg_p2p.transformation)
    print("")
    
    return reg_p2p

In [44]:
def loadPointCloud(filename):
    pcloud = np.loadtxt(filename);
    return pcloud;

In [45]:
def transform(B,T):
    Bh = np.ones((4,B.shape[0]))
    Bh[:3,:] = np.copy(B.T)
    Bh = Bh.T
    Bh = Bh @ T

    Bh[:,0] /= Bh[:,3]
    Bh[:,1] /= Bh[:,3]
    Bh[:,2] /= Bh[:,3]
    Bh[:,3] /= Bh[:,3]
    
    return Bh[:,:3]

def compute_dist(A,B,T):
    Bt = transform(B,T)
    N = Bt.shape[0]
    
    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(A)
    distances, indices = neigh.kneighbors(Bt, return_distance=True)
    return distances.ravel(), indices.ravel()

In [46]:
start = time.time()

A_ref = "data/data_bunny.txt"
B_ref = "data/data_bunny.txt"

T_iden = np.identity(4)

p2p = icp(A_ref, B_ref, T_iden)

A = loadPointCloud(A_ref)
B = loadPointCloud(B_ref)

print("Local registration took %.3f sec.\n" % (time.time() - start))

distances, indices = compute_dist(A,B,Ti)
print(np.sum(distances))


distances, indices = compute_dist(A,B,p2p.transformation)
print(np.sum(distances))


Initial alignment
registration::RegistrationResult with fitness = 1.000000, inlier_rmse = 0.000000, and correspondence_set size of 30379
Access transformation to get result.
Apply point-to-point ICP
registration::RegistrationResult with fitness = 1.000000, inlier_rmse = 0.000000, and correspondence_set size of 30379
Access transformation to get result.
Transformation is:
[[ 1.00000000e+00 -5.55111512e-17  4.16333634e-17  5.68353830e-18]
 [-8.32667268e-17  1.00000000e+00  1.11022302e-16 -1.80164581e-17]
 [ 4.16333634e-17  0.00000000e+00  1.00000000e+00 -2.77555756e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Local registration took 0.683 sec.

0.0
8.922199765492322e-12


In [47]:
corresp = np.asarray(p2p.correspondence_set)
fit = np.asarray(p2p.fitness)



print(A.shape, B.shape)

(30379, 3) (30379, 3)
